<a href="https://colab.research.google.com/github/sonalgan/DeepLearning/blob/main/Project_HPE_GRP_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

load and read the  Pre-Processed data

In [ ]:
import numpy as np
import pandas as pd
#loading the data
df=pd.read_csv('https://github.com/404S-retr0/HPE_Project_Grp-8/blob/main/Data_Wordlists.csv?raw=true')

Defining Word2Vec 

In [ ]:
#Creating dictionary of each language (English,Danish,Dutch,French,nGerman,Italian,Polish,Portuguese,Spanish,Swedish)
word2int_eng = {} #93054 words
for i in range(93056):
  word2int_eng[df.WORDS[i]] = i

word2int_dan = {} #96825 words
for i in range(93056,189881):
  word2int_dan[df.WORDS[i]] = i

word2int_dut = {} #96698 words
for i in range(189881,286579):
  word2int_dut[df.WORDS[i]] = i

word2int_fre = {} #95564 words
for i in range(286579,382144):
  word2int_fre[df.WORDS[i]] = i

word2int_ger = {} #97447 words
for i in range(382144,479592):
  word2int_ger[df.WORDS[i]] = i

word2int_ita = {} #96847 words
for i in range(479592,576439):
  word2int_ita[df.WORDS[i]] = i

word2int_pol = {} #98181 words
for i in range(576439,674620):
  word2int_pol[df.WORDS[i]] = i

word2int_por = {} #96429 words
for i in range(674620,771049):
  word2int_por[df.WORDS[i]] = i

word2int_spa = {} #97058 words
for i in range(771049,868107):
  word2int_spa[df.WORDS[i]] = i

word2int_swe = {} #96870 words
for i in range(868107,964977):
  word2int_swe[df.WORDS[i]] = i

In [ ]:
#Nesting of Dictionary 
word2int = {
  1 : word2int_eng,    #English
  2 : word2int_dan,    #Danish
  3 : word2int_dut,    #Dutch
  4 : word2int_fre,    #French
  5 : word2int_ger,    #German
  6 : word2int_ita,    #Italian
  7 : word2int_pol,    #Polish
  8 : word2int_por,    #Portuguese
  9 : word2int_spa,    #Spanish
  10 : word2int_swe    #Swedish
}
int2word = {}
for i,word in enumerate(df['WORDS'].values):
  int2word[i] = df.WORDS[i] 

df['int2word'] = int2word

Splitting Data into Train and Test sets (80:20)

In [ ]:
from sklearn.model_selection import train_test_split
x,y=df['int2word'].values,df['LANGAUAGE_VECTOR'].values
#Reshape your data using array.reshape(-1, 1) because of data has a single feature.
x=x.reshape(-1,1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(771981, 1)
(192996, 1)
(771981,)
(192996,)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Create KNN classifier Model and Fit the classifier to the data
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

Model Evaluation and trail run

In [ ]:
ac = "Accuracy Score = {}".format(knn.score(x_test, y_test))
print(ac)
tm="Testing the model\nrandomw five words location:-\n{}\nPredection of which word belong to which language belong :-{}".format(x_test[0:5],knn.predict(x_test)[0:5])
print(tm)

Accuracy Score = 0.9999844556363863
Testing the model
randomw five words location:-
[[302556]
 [140769]
 [399592]
 [555380]
 [148898]]
Predection of which word belong to which language belong :-[4 2 5 6 2]


Saving the Model 

In [ ]:
import pickle
File = open('LDPModel.pckl','wb')
pickle.dump(knn,File)
File.close()

Defining text2list function

In [ ]:
def text2list(text):
  import string
  import re
  test_str=text
  test_str = ''.join([i for i in test_str if not i.isdigit()]) 
  test_str=test_str.lower()
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~+-=|`'''

  for ele in test_str:
    if ele in punc:
      test_str = test_str.replace(ele, "")  

  test_str_split=re.split('\s+', test_str)
  
  if (test_str_split[0]==''):
    test_str_split.remove('')
  if (test_str_split[len(test_str_split)-1]==''):
    test_str_split.remove('')

  return test_str_split


Loading the Model and defining lang_detect_in_percentage function

In [ ]:
def lang_detect(text):
  list_of_word=text2list(text)

  import pickle
  global LangDetectModel
  LangDetectFile = open('LDPModel.pckl','rb')
  LangDetectModel = pickle.load(LangDetectFile)
  LangDetectFile.close();
  
  eng=0
  dan=0
  dut=0
  fre=0
  ger=0
  ita=0
  pol=0
  por=0
  spa=0
  swe=0
  unknown=0
  for q in range(len(list_of_word)):
    pos=[]
    try:
      for i in range(1,11):
        p=word2int[i][list_of_word[q]]
        pos.append(p)
    except:
      if (len(pos)==0):
        unknown=1
    else:
      for j in range(len(pos)):
        pre=LangDetectModel.predict(x)[pos[j]]
        if (pre==1):
          eng +=1
        elif (pre==2):
          dan +=1
        elif (pre==3):
          dut +=1
        elif (pre==4):
          fre +=1
        elif (pre==5):
          ger +=1
        elif (pre==6):
          ita +=1
        elif (pre==7):
          pol +=1
        elif (pre==8):
          por +=1
        elif (pre==9):
          spa +=1
        elif (pre==10):
          swe +=1
        else:
         None

  total=eng+dan+dut+fre+ger+ita+pol+por+spa+swe+unknown
  p_eng=eng/total
  p_dan=dan/total
  p_dut=dut/total
  p_fre=fre/total
  p_ger=ger/total
  p_ita=ita/total
  p_pol=pol/total
  p_por=por/total
  p_spa=spa/total
  p_swe=swe/total
  p_unk=unknown/total
  report = "Percentage Distribution\nEnglish:-{}\nDanish:-{}\nDutch:-{}\nFrench:-{}\nGerman:-{}\nItalian:-{}\nPolish:-{}\nPortuguese:-{}\nSpanish:-{}\nSwedish:-{}\nUnknown:-{}".format( p_eng,p_dan,p_dut,p_fre,p_ger,p_ita,p_pol,p_por,p_spa,p_swe,p_unk)
  print(report)

In [ ]:
lang_detect('hello @amigo , m+y name is 007. wie gehts? ¿necesitas ayuda?')

Percentage Distribution
English:-0.0975609756097561
Danish:-0.0975609756097561
Dutch:-0.0975609756097561
French:-0.0975609756097561
German:-0.0975609756097561
Italian:-0.0975609756097561
Polish:-0.0975609756097561
Portuguese:-0.0975609756097561
Spanish:-0.0975609756097561
Swedish:-0.0975609756097561
Unknown:-0.024390243902439025
